In [1]:
using SeisPlot,PyPlot

In [2]:
function cgls(A, y, x0, tol, max_iter)
# min_x ||A x-y||_2^2 

    m, n = size(A)
    x = x0
    r = A' * (y - A * x)
    p = r
    rsold = r' * r

    for i in 1:max_iter
        Ap = A * p
        alpha = rsold / (Ap' * Ap)
        x = x + alpha * p
        r = r - alpha * A' * Ap
        rsnew = r' * r
        if sqrt(rsnew) < tol
            break
        end
        p = r + (rsnew / rsold) * p
        rsold = rsnew
    end

    return x
end

cgls (generic function with 1 method)

In [108]:
function cgls_Implicit(y, x0, tol, max_iter,ntraces,s,rec,nx,nz,dx,dz,nt,dt,v)
# min_x ||A x-y||_2^2 Implicit


    x = x0
    tmp = y .- demigra(ntraces,s,rec,nx,nz,dx,dz,nt,dt,v,x)
    r = migra(ntraces,s,rec,nx,nz,dx,dz,nt,dt,v,tmp)
    p = r
    rsold = sum(r.*r)

    for i in 1:max_iter
        Ap = demigra(ntraces,s,rec,nx,nz,dx,dz,nt,dt,v,p)
        alpha = rsold / sum(Ap.* Ap)
        x = x + alpha * p
        r = r - alpha * migra(ntraces,s,rec,nx,nz,dx,dz,nt,dt,v,Ap) # 
        rsnew = r' * r
        if sqrt(rsnew) < tol
            break
        end
        p = r + (rsnew / rsold) * p
        rsold = rsnew
    end

    return x
end

cgls_Implicit (generic function with 2 methods)

In [73]:
function sd(A, y, x0, step, tol, max_iter)
    # min_x ||A x-y||_2^2 
    
        m, n = size(A)
        x = x0
    
        for i in 1:max_iter
            r = A*x-y 
            x = x - step*(A'*r)
            res= r' * r
            if sqrt(res) < tol
                break
            end
        end
    
        return x
    end

sd (generic function with 1 method)

In [87]:
# Testing CGLS with Explicit A. 
A = randn(20,10)
x = randn(10)
y = A*x+0.001*randn(20)
x0 = zeros(10)
tol = 0.0001
max_iter = 10


x1 = cgls(A, y, x0, tol, 20)
x2 = sd(A, y, x0,  0.001, 0.0, 1290)

x3 = (A'*A)\(A'y)

hcat(x1,x2,x3);



###  Dot product test to ensure migration and demigration functions behave like L' and L

Born / Kirchoff PSTM Example


In [92]:
nx = 200; nz = 200
dx = 10.; dz = 10.
nt = 1900; dt = 0.002
ntraces = 160

Lx = (nx-1)*dx
Lz = (nz-1)*dz
 v = ones(nz,nx)*1000; 

 
 

 m1 = randn(nz,nx);      d1 = demigra(ntraces,s,r,nx,nz,dx,dz,nt,dt,v,m1)
 d2 = randn(nt,ntraces); m2 =   migra(ntraces,s,r,nx,nz,dx,dz,nt,dt,v,d2)

aux1 = d1.*d2;
aux2 = m1.*m2;
println( sum(aux1[:])," ", sum(aux2[:]) )


-4915.401764273689 -4915.401764273698


### Test code with inverse problem crime. 

First we use a model to generate
data via the demigration operator. Then, we use the adjoint to recover the image

In [110]:
ns = 12  # 20 sources
 nr = 100 # 100 receivers
 ntraces = ns*nr

s = zeros(ntraces)
r = zeros(ntraces)
 
k = 1
  for is = 1:ns
    for ir = 1:nr
    s[k] = 10. + 25*(is)
    r[k] = 10. + 10.0+(ir-1)*5
k = k + 1
  end
end
m_true = zeros(nz,nx);
m_true[100,1:50].=1.0;
m_true[112,50:end].=1.0;
m_true[130,:].=-1.0;

d =     demigra(ntraces,s,r,nx,nz,dx,dz,nt,dt,v,m_true)
m_adj = migra(ntraces,s,r,nx,nz,dx,dz,nt,dt,v,d)
m_adj = m_adj/maximum(m_adj)
m_cgls= cgls_Implicit(d, zeros(size(m_true)), 0.001,3,ntraces,s,r,nx,nz,dx,dz,nt,dt,v)



#subplot(221);imshow(m_true,cmap="seismic");
#subplot(222);imshow(m_adj,cmap="seismic",vmin=-0.9,vmax=0.9);
#subplot(223);imshow(d,cmap="seismic");


LoadError: MethodError: no method matching isless(::Matrix{Float64}, ::Float64)

[0mClosest candidates are:
[0m  isless([91m::PyCall.PyObject[39m, ::Any)
[0m[90m   @[39m [33mPyCall[39m [90m~/.julia/packages/PyCall/1gn3u/src/[39m[90m[4mpyoperators.jl:75[24m[39m
[0m  isless(::Any, [91m::PyCall.PyObject[39m)
[0m[90m   @[39m [33mPyCall[39m [90m~/.julia/packages/PyCall/1gn3u/src/[39m[90m[4mpyoperators.jl:76[24m[39m
[0m  isless([91m::Missing[39m, ::Any)
[0m[90m   @[39m [90mBase[39m [90m[4mmissing.jl:87[24m[39m
[0m  ...


In [111]:
function demigra(ntraces,s,r,nx,nz,dx,dz,nt,dt,v,m)
# Program for simple demigration d = L m
    d = zeros(nt,ntraces)
 for k = 1:ntraces
    for ix = 1:nx
       for iz = 1:nz
            dr = ((ix-1)*dx-r[k])^2+((iz-1)*dz)^2; dr =sqrt(dr)
            ds = ((ix-1)*dx-s[k])^2+((iz-1)*dz)^2; ds =sqrt(ds)
            time = (dr+ds)/v[iz,ix]
            it = floor(Int,time/dt+1)
            if (  it<nt )
               d[it,k] = d[it,k] + m[iz,ix]
            end
        end
    end
end
return d
end

function migra(ntraces,s,r,nx,nz,dx,dz,nt,dt,v,d)
# Program for simple migration ma = L' m
    ma = zeros(nz,nx)
 for k = 1:ntraces
    for ix = 1:nx
       for iz = 1:nz
            dr = ((ix-1)*dx-r[k])^2+((iz-1)*dz)^2; dr =sqrt(dr)
            ds = ((ix-1)*dx-s[k])^2+((iz-1)*dz)^2; ds =sqrt(ds)
            time = (dr+ds)/v[iz,ix]
            it = floor(Int,time/dt+1)  # Assigned nearest integer 
            
            if (  it<nt )
             
         ma[iz,ix] = ma[iz,ix] +  d[it,k]### Emepezar aqui --> luego ir al adj
            end
        end
    end
end
    return ma
end


migra (generic function with 1 method)